# 데이터 load

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm  # 진행률 표시
import os

In [2]:
filepath='../../data/'

In [3]:
painting_lst = np.load(os.path.join(filepath,'data.npy'))
painting_lst.shape

(80158, 128, 128, 3)

In [4]:
# 1️⃣ EfficientNetB0 모델 로드 (Fully Connected Layer 제외)
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
model = Model(inputs=base_model.input, outputs=tf.keras.layers.GlobalAveragePooling2D()(base_model.output))

def extract_features_batch(image_batch):
    """이미지 배치를 받아 특징 벡터 추출"""
    image_batch = preprocess_input(image_batch)  # EfficientNet 전처리 적용
    features = model.predict(image_batch, verbose=0)  # 특징 벡터 추출
    return features  # shape: (batch_size, 1280)

In [5]:
# 2️⃣ 데이터셋 로드 (메모리 절약을 위해 float32 변환)
painting_lst = np.array(painting_lst, dtype=np.float32)  # float64 → float32 변환
num_samples = painting_lst.shape[0]

In [6]:
# 3️⃣ 배치 단위로 특징 벡터 추출
batch_size = 128  # 메모리 부담을 줄이기 위해 조정
feature_list = []

for i in tqdm(range(0, num_samples, batch_size), desc="Extracting Features"):
    batch = painting_lst[i:i+batch_size]  # 배치 단위로 데이터 가져오기
    features = extract_features_batch(batch)  # 특징 벡터 추출
    feature_list.append(features)

Extracting Features: 100%|███████████████████████████████████████████████████████████| 627/627 [00:56<00:00, 11.06it/s]


In [7]:
# 4️⃣ 모든 특징 벡터를 하나의 배열로 결합
feature_vectors = np.vstack(feature_list)  # 최종 결과 (80158, 1280)
print("Final Feature Vector Shape:", feature_vectors.shape)  # (80158, 1280)

Final Feature Vector Shape: (80158, 1280)


In [8]:
np.save(os.path.join(filepath,'EfficientNetB0_vectors.npy'),feature_vectors)